In [1]:
from typing import Any, Type
from langchain_openai import ChatOpenAI
from langchain_community.retrievers import WikipediaRetriever
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from langchain_community.retrievers import WikipediaRetriever
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import BaseTool
import requests
from bs4 import BeautifulSoup
import openai as client
import json

model = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    temperature=0.1,
)

def wikipedia_search_tool(inputs):
    wk = WikipediaSearchTool()
    query=inputs["query"]
    result = wk.run(query)
    return json.dumps({"result": result}, ensure_ascii=False)

def duckduckgo_search_tool(inputs):
    ddg = DuckDuckGoSearchTool()
    query=inputs["query"]
    result=ddg.run(query)
    return json.dumps({"result": result})

def save_file(docs, title):
    f = open(f"./{title}.txt", "w", encoding="utf-8")
    f.write(docs)


def search_wikipedia(keyword):
    result = "Wikipedia Result\n\n"
    retriver = WikipediaRetriever(top_k_results=3, lang="ko")
    data_list = retriver.invoke(keyword)
    for page_content in data_list:
        result += f"{page_content.page_content} \n\n"
    return result


class WikipediaToolArgsSchema(BaseModel):
    query: str = Field(description="The query you want to search on Wikipedia")


class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    Use this tool to search information on Wikipedia. Use 'keyword' as a parameter.
    """
    args_schema: Type[WikipediaToolArgsSchema] = WikipediaToolArgsSchema

    def _run(self, query: str, **kwargs: Any):
        wiki = search_wikipedia(query)
        save_file(wiki, "wiki")
        return wiki


class DuckDuckGoToolArgsCshema(BaseModel):
    query: str = Field(description="The query you want to search on DuckDuckGo")


class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = """
    Use this tool to search information on DuckDuckGo. Use 'keyword' as a parameter.  
    """
    args_schema: Type[DuckDuckGoToolArgsCshema] = DuckDuckGoToolArgsCshema

    def _scrape_website(self, url: str):
        """Scrapes content from the given URL."""
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            page_text = soup.get_text(separator="\n")
            return page_text
        else:
            return f"Failed to access {url}"

    def _run(self, query: str, **kwargs: Any):
        ddg = DuckDuckGoSearchAPIWrapper()
        result = ddg.results(query, max_results=3)
        print(result)
        if result and len(result) > 0:
            first_link = result[0]['link']
            scraped_content = self._scrape_website(first_link)
            save_file(scraped_content, "ddg_scraped_content")
            return scraped_content
        else:
            return "No results found or no website link found in DuckDuckGo search."

functions_map = {
    "wikipedia_search_tool": wikipedia_search_tool,
    "duckduckgo_search_tool": duckduckgo_search_tool,
}

functions=[
  {
    "type": "function",
    "function": {
      "name": "wikipedia_search_tool",
      "description": "Use this tool to search information on Wikipedia. Use 'query' as a parameter.",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "The query you want to search on Wikipedia"
          },
        },
        "required": ["query"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "duckduckgo_search_tool",
      "description": "Use this tool to search information on DuckDuckGo. Use 'query' as a parameter.",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "The query you want to search on DuckDuckGo"
          },
        },
        "required": ["query"]
      }
    }
  },
]


assistant = client.beta.assistants.create(
  name="Research Assistant",
  instructions=(
        "You are a helpful assistant that uses provided tools to answer the user's questions. "
        "When a user asks for information, you should use the appropriate tool to find the information. "
        "Use the 'wikipedia_search_tool' or 'duckduckgo_search_tool' as needed. "
        "If the results contain a URL, extract the content from the link."
    ),
  model="gpt-4o-mini-2024-07-18",
  tools=functions,
)

assistant_id = "asst_HyD9htYlSvYHADUcbTxNLT8L"


thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Giant Panda에 대해 알려주세요.",
        }
    ] 
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant_id,
)

def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []

    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outpus
    )




In [2]:
wikipedia_search_tool({"query": "Giant Panda"})

'{"result": "Wikipedia Result\\n\\n대왕판다(大王판다, 문화어: 참대곰, giant panda, 중국어:大熊猫, 학명: Ailuropoda melanoleuca)는 중국 쓰촨성 지방과 티베트의 고산 지대에 서식하는 곰과 포유동물이다. 자이언트판다, 자이언트를 생략하고 판다라고 하기도 하고 왕판다라고도 한다. 영어식 발음으로 팬더라고도 하나, 표준어로 인정하는 표기는 판다이다. 영어권에서 쓰는 이름인 ‘giant panda’는 애기판다처럼 대나무잎을 먹는 큰 동물이라는 점 때문에 붙였다. 중국어에서 애기판다를 작은 대왕판다라는 뜻의 ‘샤오슝마오(중국어: 小熊猫 ‘소웅묘’[*])’라고 부른다. 대한민국에서는 자생하지는 않으나 동물원에 4마리가 있다.\\n천적은 눈표범, 자칼, 담비다.\\n\\n\\n== 생태 ==\\n큰 덩치, 귀와 눈 주위의 검은 반점으로 눈에 쉽게 띄인다. 식육목에 속하지만, 먹이 99%는 대나무다. 가끔씩 야생 판다는 다른 식물이나 육류를 섭취하기도 한다.\\n대왕 판다는 주로 쓰촨성 지방의 산간 지역에 서식하고, 산시성과 간쑤성에도 서식한다.\\n개간, 벌채 등의 인간 활동으로 판다 서식지가 매년 줄어든다.\\n판다는 멸종위기종이었다. 2007년, 중국과 다른 27개의 국가에서 239마리를 사육하였고 (야생 판다 개체 수는 자료마다 다르다. 어떤 곳은 1,590마리 로 추산하는 반면, 2006년 연구에 2,000마리에서 3,000마리 가량이라는 결과가 있다.) 또, 개체 수가 증가 추세라는 보고도 있다. 그러나 IUCN은 등급을 격하할 만큼 충분하지는 않다고 하였으나 2016년 9월 4일 IUCN은 2014년 성체 기준 1864마리로 증가했으므로 멸종위기종에서 멸종취약종으로 멸종위기등급을 한 단계 낮추었다.\\n세계자연기금의 상징이다. 보통 중국내에서 중국을 상징하는 동물은 용이지만, 국제적으로 판다가 대표한다. 그 예로 2008년 하계 올림픽 마스코트인 푸와가 있으며, 1990년 아시안 게임 마스코트 역시 판다

In [3]:
duckduckgo_search_tool({"query": "Giant Panda"})

[{'snippet': 'Learn about the giant panda, a black-and-white bear that lives in bamboo forests in China and eats mostly bamboo. Find out how pandas use scent, vocalizations, and gestation to communicate and reproduce.', 'title': 'Giant panda | Facts, Habitat, Population, & Diet | Britannica', 'link': 'https://www.britannica.com/animal/giant-panda'}, {'snippet': "Two new giant pandas will be arriving at the Smithsonian's National Zoo in Washington, DC, this year, the zoo announced Wednesday. The bears, named Qing Bao and Bao Li, a pair of 2-year-old ...", 'title': 'Giant pandas returning to the National Zoo in DC | CNN', 'link': 'https://www.cnn.com/2024/05/29/us/giant-pandas-return-to-national-zoo/index.html'}, {'snippet': "CNN —. San Diego's newest giant pandas are acclimating well to their new state-side home, the San Diego Zoo Wildlife Alliance said Tuesday. The zoo shared first-look photos of the pair, Yun ...", 'title': 'San Diego Zoo shares first look at new giant pandas followin

'{"result": "\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\nGiant panda | Facts, Habitat, Population, & Diet | Britannica\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\nSearch Britannica\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\nClick here to search\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\nSearch Britannica\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\nClick here to search\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n   Subscribe\\n\\n\\n\\n\\n\\n\\n   Subscribe\\n\\n\\n\\n\\n\\nLogin\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\nHome\\n\\n\\nHistory & Society\\n\\n\\nScience & Tech\\n\\n\\nBiographies\\n\\n\\nAnimals & Nature\\n\\n\\nGe

In [ ]:
send_message(thread.id, "Giant Panda에 대해 알려주세요.")

In [ ]:
get_messages(thread.id)